In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [18]:
!pwd

/home/jupyter/demos/vertexai/matching-engine


<table align="left">

  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/matching_engine/matching_engine_for_indexing.ipynb">
      Run in Google Cloud Notebooks
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/matching_engine/matching_engine_for_indexing.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This example demonstrates how to use the GCP ANN Service. It is a high scale, low latency solution, to find similar vectors (or more specifically "embeddings") for a large corpus. Moreover, it is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

### Dataset

The dataset used for this tutorial is the [GloVe dataset](https://nlp.stanford.edu/projects/glove/).

### Objective

In this notebook, you will learn how to create Approximate Nearest Neighbor (ANN) Index, query against indexes, and validate the performance of the index. 

The steps performed include:

* Create ANN Index and Brute Force Index
* Create an IndexEndpoint with VPC Network
* Deploy ANN Index and Brute Force Index
* Perform online query
* Compute recall


### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

* **Prepare a VPC network**.  To reduce any network overhead that might lead to unnecessary increase in overhead latency, it is best to call the ANN endpoints from your VPC via a direct [VPC Peering](https://cloud.google.com/vertex-ai/docs/general/vpc-peering) connection. The following section describes how to setup a VPC Peering connection if you don't have one. This is a one-time initial setup task. You can also reuse existing VPC network and skip this section.
* **WARNING:** The match service gRPC API (to create online queries against your deployed index) has to be executed in a Google Cloud Notebook instance that is created with the following requirements:
  * **In the same region as where your ANN service is deployed** (for example, if you set `REGION = "us-central1"` as same as the tutorial, the notebook instance has to be in `us-central1`).
  * **Make sure you select the VPC network you created for ANN service** (instead of using the "default" one). That is, you will have to create the VPC network below and then create a new notebook instance that uses that VPC.  
  * If you run it in the colab or a Google Cloud Notebook instance in a different VPC network or region, the gRPC API will fail to peer the network (InactiveRPCError).

In [ ]:
PROJECT_ID = "living-media-extraction-poc"  # @param {type:"string"}
NETWORK_NAME = "ucaip-haystack-vpc-network"  # @param {type:"string"}
PEERING_RANGE_NAME = "ucaip-haystack-range"

# Create a VPC network
! gcloud compute networks create {NETWORK_NAME} --bgp-routing-mode=regional --subnet-mode=auto --project={PROJECT_ID}

# Add necessary firewall rules
! gcloud compute firewall-rules create {NETWORK_NAME}-allow-icmp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow icmp

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-internal --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow all --source-ranges 10.128.0.0/9

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-rdp --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:3389

! gcloud compute firewall-rules create {NETWORK_NAME}-allow-ssh --network {NETWORK_NAME} --priority 65534 --project {PROJECT_ID} --allow tcp:22

# Reserve IP range
! gcloud compute addresses create {PEERING_RANGE_NAME} --global --prefix-length=16 --network={NETWORK_NAME} --purpose=VPC_PEERING --project={PROJECT_ID} --description="peering range for uCAIP Haystack."

# Set up peering with service networking
! gcloud services vpc-peerings connect --service=servicenetworking.googleapis.com --network={NETWORK_NAME} --ranges={PEERING_RANGE_NAME} --project={PROJECT_ID}

In [1]:
PROJECT_ID = "on-prem-project-337210"
NETWORK_NAME = "ucaip-haystack-vpc-network"
PEERING_RANGE_NAME = "ucaip-haystack-range"

* Authentication: `$ gcloud auth login` rerun this in Google Cloud Notebook terminal when you are logged out and need the credential again.

### Installation

Download and install the latest (preview) version of the Vertex SDK for Python.

In [ ]:
! pip install -U git+https://github.com/googleapis/python-aiplatform.git@main-test --user

Install the `h5py` to prepare sample dataset, and the `grpcio-tools` for querying against the index. 

In [ ]:
! pip install -U grpcio-tools --user
! pip install -U h5py --user

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [4]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager).

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API, and Service Networking API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,servicenetworking.googleapis.com).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [2]:
import os

PROJECT_ID = "on-prem-project-337210"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  on-prem-project-337210


Otherwise, set your project ID here.

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with Vertex AI.

In [3]:
BUCKET_NAME = "gs://customer-demos/matching-engine"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [4]:
! gsutil ls -al $BUCKET_NAME

         0  2022-05-11T12:26:39Z  gs://customer-demos/matching-engine/#1652271999637434  metageneration=1
     25547  2022-05-15T16:56:04Z  gs://customer-demos/matching-engine/vertex_face_embeddings.json#1652633764938247  metageneration=1
TOTAL: 2 objects, 25547 bytes (24.95 KiB)


### Import libraries and define constants

In [5]:
!ls

__pycache__
dataset
delete_Indexes_indexendpoints.ipynb
face
face_alignment_step1.py
face_alignment_step2.py
face_detection.npz
face_input_generate_embeddings_step4.ipynb
face_vertexai_match_create_index_step3.ipynb
faces-embeddings.npz
match_service.proto
model
request.json
third_party
vertex_face_embeddings.json


Import the Vertex AI (unified) client library into your Python environment. 


In [6]:
import time

import grpc
import h5py
from google.cloud import aiplatform_v1beta1
from google.protobuf import struct_pb2

In [7]:
REGION = "us-central1"
ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
NETWORK_NAME = "ucaip-haystack-vpc-network"  # @param {type:"string"}


AUTH_TOKEN = !gcloud auth print-access-token
PROJECT_NUMBER = !gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = PROJECT_NUMBER[0]

PARENT = "projects/{}/locations/{}".format(PROJECT_ID, REGION)

print("ENDPOINT: {}".format(ENDPOINT))
print("PROJECT_ID: {}".format(PROJECT_ID))
print("REGION: {}".format(REGION))

!gcloud config set project {PROJECT_ID}
!gcloud config set ai_platform/region {REGION}

ENDPOINT: us-central1-aiplatform.googleapis.com
PROJECT_ID: on-prem-project-337210
REGION: us-central1
Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

Updated property [ai_platform/region].


## Prepare the Data

The GloVe dataset consists of a set of pre-trained embeddings. The embeddings are split into a "train" split, and a "test" split.
We will create a vector search index from the "train" split, and use the embedding vectors in the "test" split as query vectors to test the vector search index.

NOTE: While the data split uses the term "train", these are pre-trained embeddings and thus are ready to be indexed for search. The terms "train" and "test" split are used just to be consistent with usual machine learning terminology.

Download the GloVe dataset.


Read the data into memory.


In [8]:
# # The number of nearest neighbors to be retrieved from database for each query.
# k = 10

# h5 = h5py.File(""glove-100-angular.hdf5, "r")
# train = h5["train"]
# test = h5["test"]

import numpy as np
k=4
# load dataset
data = np.load('faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))
print('Dataset: train labels=%d, test labels=%d' % (trainy.shape[0], testy.shape[0]))


print('Dataset: train labels=%s' % (trainy[0]))#######
print("total train embeddings:",len(trainX))
print("total train labels:",len(trainy))

faces_dataset_numpy =trainX
faces_dataset_numpy.shape
#print(faces_dataset_numpy[0])
test_query_label=trainy

Dataset: train=18, test=3
Dataset: train labels=18, test labels=3
Dataset: train labels=Sachin
total train embeddings: 18
total train labels: 18


Save the train split in JSONL format.


In [9]:
with open("vertex_face_embeddings.json", "w") as f:
    for i in range(len(trainX)):
        f.write('{"id":"' + trainy[i] + "_" + str(i) + '",')
        f.write('"embedding":[' + ",".join(str(x) for x in faces_dataset_numpy[i]) + "]}")
        f.write("\n")

Upload the training data to GCS.

In [10]:
! gsutil cp vertex_face_embeddings.json {BUCKET_NAME}/vertex_face_embeddings.json

Copying file://vertex_face_embeddings.json [Content-Type=application/json]...
/ [1 files][ 25.0 KiB/ 25.0 KiB]                                                
Operation completed over 1 objects/25.0 KiB.                                     


In [11]:
! gsutil ls {BUCKET_NAME}

gs://customer-demos/matching-engine/
gs://customer-demos/matching-engine/vertex_face_embeddings.json


## Create Indexes


### Create ANN Index (for Production Usage)

In [12]:
index_client = aiplatform_v1beta1.IndexServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

In [13]:
DIMENSIONS = 128
DISPLAY_NAME = "vertexai_face_search_demo"
#DISPLAY_NAME_BRUTE_FORCE = DISPLAY_NAME + "_brute_force"

Create the ANN index configuration:

Please read the documentation to understand the various configuration parameters that can be used to tune the index


In [14]:
treeAhConfig = struct_pb2.Struct(
    fields={
        "leafNodeEmbeddingCount": struct_pb2.Value(number_value=500),
        "leafNodesToSearchPercent": struct_pb2.Value(number_value=7),
    }
)

algorithmConfig = struct_pb2.Struct(
    fields={"treeAhConfig": struct_pb2.Value(struct_value=treeAhConfig)}
)

config = struct_pb2.Struct(
    fields={
        "dimensions": struct_pb2.Value(number_value=DIMENSIONS),
        "approximateNeighborsCount": struct_pb2.Value(number_value=150),
        "distanceMeasureType": struct_pb2.Value(string_value="SQUARED_L2_DISTANCE"),
        "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
    }
)

metadata = struct_pb2.Struct(
    fields={
        "config": struct_pb2.Value(struct_value=config),
        "contentsDeltaUri": struct_pb2.Value(string_value=BUCKET_NAME),
    }
)

vertex_index_matching_lmedia = {
    "display_name": DISPLAY_NAME,
    "description": "living 100 ANN index",
    "metadata": struct_pb2.Value(struct_value=metadata),
}

In [15]:
vertex_index_matching_lmedia = index_client.create_index(parent=PARENT, index=vertex_index_matching_lmedia)

In [ ]:
!ls

In [ ]:
# Poll the operation until it's done successfullly.
# This will take ~45 min.

while True:
    if vertex_index_matching_lmedia.done():
        break
    print("Poll the operation to create index...")
    time.sleep(60)
print("done")

Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...
Poll the operation to create index...


In [18]:
INDEX_RESOURCE_NAME = vertex_index_matching_lmedia.result().name
#INDEX_RESOURCE_NAME = 'projects/533775010529/locations/us-central1/indexes/9173920401883922432'

## Update Indexes

Create incremental data file.


In [24]:
with open("face_embeddings.json", "w") as f:
    f.write(
        '{"id":"0","embedding":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}\n'
    )

Copy the incremental data file to a new subdirectory.


In [25]:
! gsutil cp glove100_incremental.json {BUCKET_NAME}/incremental/glove100.json

Copying file://glove100_incremental.json [Content-Type=application/json]...
/ [1 files][  225.0 B/  225.0 B]                                                
Operation completed over 1 objects/225.0 B.                                      


Create update index request


In [26]:
metadata = struct_pb2.Struct(
    fields={
        "contentsDeltaUri": struct_pb2.Value(string_value=BUCKET_NAME + "/incremental"),
    }
)

ann_index = {
    "name": INDEX_RESOURCE_NAME,
    "display_name": DISPLAY_NAME,
    "description": "Glove 100 ANN index",
    "metadata": struct_pb2.Value(struct_value=metadata),
}

In [27]:
ann_index_matching_lmedia = index_client.update_index(index=ann_index_matching_lmedia)

In [ ]:
# Poll the operation until it's done successfullly.
# This will take ~45 min.

while True:
    if vertex_index_matching_lmedia.done():
        break
    print("Poll the operation to update index...")
    time.sleep(60)

In [51]:
#INDEX_RESOURCE_NAME = vertex_index_matching_lmedia.result().name
INDEX_RESOURCE_NAME = 'projects/533775010529/locations/us-central1/indexes/5016886839588421632'

## Create an IndexEndpoint with VPC Network

In [19]:
index_endpoint_client = aiplatform_v1beta1.IndexEndpointServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

In [20]:
VPC_NETWORK_NAME = "projects/{}/global/networks/{}".format(PROJECT_NUMBER, NETWORK_NAME)
VPC_NETWORK_NAME

'projects/533775010529/global/networks/ucaip-haystack-vpc-network'

In [21]:
index_endpoint = {
    "display_name": "vertex_index_endpoint_for_demo",
    "network": VPC_NETWORK_NAME,
}

In [22]:
r = index_endpoint_client.create_index_endpoint(
    parent=PARENT, index_endpoint=index_endpoint
)

In [23]:
r.result()

name: "projects/533775010529/locations/us-central1/indexEndpoints/2078288082729172992"

In [24]:
INDEX_ENDPOINT_NAME = r.result().name
INDEX_ENDPOINT_NAME

#INDEX_ENDPOINT_NAME = 'projects/533775010529/locations/us-central1/indexEndpoints/4976354442942087168'


'projects/533775010529/locations/us-central1/indexEndpoints/2078288082729172992'

## Deploy Indexes

### Deploy ANN Index

In [25]:
DEPLOYED_INDEX_ID = "ann_deployed_new_multiple_entities"

In [26]:
deploy_ann_index = {
    "id": DEPLOYED_INDEX_ID,
    "display_name": DEPLOYED_INDEX_ID,
    "index": INDEX_RESOURCE_NAME,
}

In [27]:
r = index_endpoint_client.deploy_index(
    index_endpoint=INDEX_ENDPOINT_NAME, deployed_index=deploy_ann_index
)

In [28]:
# Poll the operation until it's done successfullly.

while True:
    if r.done():
        break
    print("Poll the operation to deploy index...")
    time.sleep(60)
print("done")

Poll the operation to deploy index...
Poll the operation to deploy index...
Poll the operation to deploy index...
Poll the operation to deploy index...
done


In [29]:
r.result()

deployed_index {
  id: "ann_deployed_new_multiple_entities"
}

## Create Online Queries

After you built your indexes, you may query against the deployed index through the online querying gRPC API (Match service) within the virtual machine instances from the same region (for example 'us-central1' in this tutorial).  

The way a client uses this gRPC API is by folowing steps:

* Write `match_service.proto` locally
* Clone the repository that contains the dependencies of match_service.proto in the Terminal:

`$ mkdir third_party && cd third_party`

`$ git clone https://github.com/googleapis/googleapis.git`

* Compile the protocal buffer (see below)
* Obtain the index endpoint
* Use a code-generated stub to make the call, passing the parameter values

In [30]:
%%writefile match_service.proto

syntax = "proto3";

package google.cloud.aiplatform.container.v1beta1;

import "google/rpc/status.proto";

// MatchService is a Google managed service for efficient vector similarity
// search at scale.
service MatchService {
  // Returns the nearest neighbors for the query. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc Match(MatchRequest) returns (MatchResponse) {}

  // Returns the nearest neighbors for batch queries. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc BatchMatch(BatchMatchRequest) returns (BatchMatchResponse) {}
}

// Parameters for a match query.
message MatchRequest {
  // The ID of the DeploydIndex that will serve the request.
  // This MatchRequest is sent to a specific IndexEndpoint of the Control API,
  // as per the IndexEndpoint.network. That IndexEndpoint also has
  // IndexEndpoint.deployed_indexes, and each such index has an
  // DeployedIndex.id field.
  // The value of the field below must equal one of the DeployedIndex.id
  // fields of the IndexEndpoint that is being called for this request.
  string deployed_index_id = 1;

  // The embedding values.
  repeated float float_val = 2;

  // The number of nearest neighbors to be retrieved from database for
  // each query. If not set, will use the default from
  // the service configuration.
  int32 num_neighbors = 3;

  // The list of restricts.
  repeated Namespace restricts = 4;

  // Crowding is a constraint on a neighbor list produced by nearest neighbor
  // search requiring that no more than some value k' of the k neighbors
  // returned have the same value of crowding_attribute.
  // It's used for improving result diversity.
  // This field is the maximum number of matches with the same crowding tag.
  int32 per_crowding_attribute_num_neighbors = 5;

  // The number of neighbors to find via approximate search before
  // exact reordering is performed. If not set, the default value from scam
  // config is used; if set, this value must be > 0.
  int32 approx_num_neighbors = 6;

  // The fraction of the number of leaves to search, set at query time allows
  // user to tune search performance. This value increase result in both search
  // accuracy and latency increase. The value should be between 0.0 and 1.0. If
  // not set or set to 0.0, query uses the default value specified in
  // NearestNeighborSearchConfig.TreeAHConfig.leaf_nodes_to_search_percent.
  int32 leaf_nodes_to_search_percent_override = 7;
}

// Response of a match query.
message MatchResponse {
  message Neighbor {
    // The ids of the matches.
    string id = 1;

    // The distances of the matches.
    double distance = 2;
  }
  // All its neighbors.
  repeated Neighbor neighbor = 1;
}

// Parameters for a batch match query.
message BatchMatchRequest {
  // Batched requests against one index.
  message BatchMatchRequestPerIndex {
    // The ID of the DeploydIndex that will serve the request.
    string deployed_index_id = 1;

    // The requests against the index identified by the above deployed_index_id.
    repeated MatchRequest requests = 2;

    // Selects the optimal batch size to use for low-level batching. Queries
    // within each low level batch are executed sequentially while low level
    // batches are executed in parallel.
    // This field is optional, defaults to 0 if not set. A non-positive number
    // disables low level batching, i.e. all queries are executed sequentially.
    int32 low_level_batch_size = 3;
  }

  // The batch requests grouped by indexes.
  repeated BatchMatchRequestPerIndex requests = 1;
}

// Response of a batch match query.
message BatchMatchResponse {
  // Batched responses for one index.
  message BatchMatchResponsePerIndex {
    // The ID of the DeployedIndex that produced the responses.
    string deployed_index_id = 1;

    // The match responses produced by the index identified by the above
    // deployed_index_id. This field is set only when the query against that
    // index succeed.
    repeated MatchResponse responses = 2;

    // The status of response for the batch query identified by the above
    // deployed_index_id.
    google.rpc.Status status = 3;
  }

  // The batched responses grouped by indexes.
  repeated BatchMatchResponsePerIndex responses = 1;
}

// Namespace specifies the rules for determining the datapoints that are
// eligible for each matching query, overall query is an AND across namespaces.
message Namespace {
  // The string name of the namespace that this proto is specifying,
  // such as "color", "shape", "geo", or "tags".
  string name = 1;

  // The allowed tokens in the namespace.
  repeated string allow_tokens = 2;

  // The denied tokens in the namespace.
  // The denied tokens have exactly the same format as the token fields, but
  // represents a negation. When a token is denied, then matches will be
  // excluded whenever the other datapoint has that token.
  //
  // For example, if a query specifies {color: red, blue, !purple}, then that
  // query will match datapoints that are red or blue, but if those points are
  // also purple, then they will be excluded even if they are red/blue.
  repeated string deny_tokens = 3;
}

Writing match_service.proto


Compile the protocol buffer, and then `match_service_pb2.py` and `match_service_pb2_grpc.py` are generated.

In [60]:
! python -m grpc_tools.protoc -I=. --proto_path=third_party/googleapis --python_out=. --grpc_python_out=. match_service.proto

Obtain the Private Endpoint: 

In [31]:
DEPLOYED_INDEX_SERVER_IP = (
    list(index_endpoint_client.list_index_endpoints(parent=PARENT))[0]
    .deployed_indexes[0]
    .private_endpoints.match_grpc_address
)
DEPLOYED_INDEX_SERVER_IP

'10.79.0.5'

Test your query:

In [70]:
import time
import grpc
from google.cloud import aiplatform_v1beta1
from google.protobuf import struct_pb2
import match_service_pb2
import match_service_pb2_grpc
DEPLOYED_INDEX_SERVER_IP='10.79.0.5'
DEPLOYED_INDEX_ID="ann_deployed_new_latest"
channel = grpc.insecure_channel("{}:10000".format(DEPLOYED_INDEX_SERVER_IP))
stub = match_service_pb2_grpc.MatchServiceStub(channel)


In [ ]:
!pip install mtcnn

In [71]:
# Test query
query = [ 0.47638232,  0.7031913,  -0.09426352, -1.6759269,  -1.5161405,   2.2671187,
  0.23024209,  1.3213857,   0.46947488, -0.65484554, -0.349672,   -0.35069403,
  0.44866818, -0.06713469, -2.0380442,   0.39039677,  1.1918312,  -0.19237979,
  1.8078632,   1.7020371,   1.0240681,   1.4708706,  -1.0571423,   0.40675738,
 -1.4315073,   0.4660923,  -0.68549013, -2.0540395,  -0.49951416, -0.12328261,
 -1.4455066,  -0.18856564, -0.41942516, -0.0844005,   0.28571644, -0.4450052,
 -0.76744795,  0.6879673,   1.7383614,  -1.0293287,   0.45323,    -1.0447822,
 -0.343994,   -0.10894796,  0.8267744,  -2.078393,    1.2808806,  -0.1248285,
  1.5109948,  -0.22050345,  1.0471966,   1.4740075,  -0.9624044,  -3.180627,
 -1.1465548,   0.74758,    -0.02445646, -1.089226,   -0.45088083,  1.8180914,
 -0.18359527, -0.8558626,   1.3302065,  -0.16613738,  0.7130253,  -0.69063926,
  1.2394042,  -0.96780926,  0.8238277,   0.1764413,   0.9056591,  -0.22822233,
 -0.3942813,   0.38060862,  1.6349559,  -1.8648696,  -1.6389544,  -1.4473922,
  0.15770671, -0.8820905,   1.0080476,  -0.4136618,   1.2733704,   1.3242897,
  0.515979,    1.9586087,   0.90002733, -1.2243577,  -1.3745974,   0.30756918,
 -0.56663024,  0.81641173,  0.25212407,  0.49709347, -0.7454099,  -0.01047975,
  0.7163955,  -0.39616713, -0.42718938,  1.220137,    0.00384053, -0.3365608,
  0.34905386,  1.752177,   -1.8956761,  -1.6031564,  -0.7969713,   0.941522,
 -1.9197286,  -0.01558837,  0.5916418,  -0.7088306,   1.4532896,   1.6064367,
 -0.329077,   -2.5688303,   0.8240242,   1.7159567,  -0.16501185, -0.27374515,
  0.5976707,   0.6173911,  -1.3319573,   1.5971177,  -3.0979125,   0.5034129
 -1.7959037,  -1.5725234, ]

request = match_service_pb2.MatchRequest()
request.deployed_index_id = DEPLOYED_INDEX_ID
for val in query:
    request.float_val.append(val)

response = stub.Match(request)
response

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1652362373.040949108","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":3217,"referenced_errors":[{"created":"@1652362373.040948042","description":"failed to connect to all addresses","file":"src/core/lib/transport/error_utils.cc","file_line":165,"grpc_status":14}]}"
>

In [65]:
print(type(response))



<class 'match_service_pb2.MatchResponse'>


In [53]:
# Test query embeddings for amith shah, which is not available in database
query = [
    1.495779,-0.66170406,-0.14169529,-0.30339828,-0.5435338,0.50911
,1.3568522,-0.4374229,0.90619797,1.6209644,-0.19036846,0.24534035
,0.4321385,0.4752176,-0.6049675,-0.7750518,0.45845032,-1.3981206
,1.0402417,-0.48912388,0.49328068,-1.1898837,-2.2606947,1.3843943
,0.94149035,1.0844613,2.0339272,-1.3142763,0.84153384,0.6325912
,1.1940684,0.89055884,0.60003215,-0.9711699,-1.2689146,0.1638731
,-1.4111251,0.02936853,0.96740484,0.6170994,1.2337803,1.0676495
,1.7319858,0.29178783,0.71463686,1.4405005,-2.2085824,0.46231887
,0.1541693,-0.1351457,0.01236704,0.70565826,-0.79534984,-1.4985534
,0.2971816,1.2317296,1.152956,-1.0975003,0.8770176,-0.7808464
,-1.0577081,1.1816063,0.23468532,0.3126467,0.04684041,0.7092123
,-1.2087427,-1.1444596,-0.83589107,-0.29516923,0.9114639,-0.27598554
,1.485636,-0.3815856,0.6790652,-1.6971464,-0.80177593,-0.957147
,-0.99157494,1.1224035,0.32742518,0.7895254,0.70823455,-1.7406285
,1.1113447,0.2233966,-1.4414437,1.9586837,0.23336703,0.96924466
,-0.52088,1.9744349,1.4874448,2.038791,1.0142369,0.11431214
,0.7924406,0.90831774,-1.6923523,0.06625814,0.49092048,-1.5426992
,1.4604397,-0.8287355,-1.411466,-1.1012169,0.860731,2.1464238
,0.07446912,-0.15823117,1.0608367,-0.88807094,0.27776843,-0.04896355
,0.14217156,-0.0058278,1.6506175,2.1949034,1.9302653,-0.8565916
,1.6819628,0.3976161,1.1564932,-0.81693643,-0.7882204,-0.10244952
,-0.19493397,-0.4020826,
    
]

request = match_service_pb2.MatchRequest()
request.deployed_index_id = DEPLOYED_INDEX_ID
for val in query:
    request.float_val.append(val)

response = stub.Match(request)
response

neighbor {
  id: "Amit Shah_32"
  distance: 2.6977949142456055
}
neighbor {
  id: "Amit Shah_51"
  distance: 26.50396156311035
}
neighbor {
  id: "Amit Shah_60"
  distance: 28.669837951660156
}
neighbor {
  id: "Amit Shah_18"
  distance: 30.226905822753906
}
neighbor {
  id: "Amit Shah_41"
  distance: 31.619911193847656
}
neighbor {
  id: "Amit Shah_28"
  distance: 33.00728225708008
}
neighbor {
  id: "Amit Shah_88"
  distance: 33.01567077636719
}
neighbor {
  id: "Amit Shah_14"
  distance: 33.430824279785156
}
neighbor {
  id: "Amit Shah_24"
  distance: 33.62293243408203
}
neighbor {
  id: "Amit Shah_25"
  distance: 34.01750946044922
}

In [14]:
print(type(response))

<class 'match_service_pb2.MatchResponse'>


In [5]:
# mask amith shah
query = [
    1.18488804e-01,2.89984912e-01,5.66803455e-01,-1.00210452e+00
,1.32127494e-01,4.29663152e-01,1.00202608e+00,-1.24064720e+00
,-1.28067121e-01,-5.70271075e-01,-8.11766088e-02,1.19131732e+00
,2.13471487e-01,5.97652555e-01,-1.88465452e+00,-6.10714734e-01
,-1.87095255e-03,1.53546765e-01,1.09359741e+00,-8.14708591e-01
,-5.40782928e-01,3.61184120e-01,-3.63611126e+00,-7.70633444e-02
,-6.55305445e-01,3.49672765e-01,-6.57574058e-01,-1.33100450e+00
,-8.76049638e-01,-1.12344635e+00,7.99252391e-01,4.43308413e-01
,4.36599016e-01,-2.01534152e+00,-4.34192300e-01,9.84311879e-01
,1.14407051e+00,8.93010944e-03,2.43781114e+00,5.30850068e-02
,1.73475587e+00,1.54777563e+00,2.96857119e-01,8.06422114e-01
,1.56641260e-01,5.00007272e-01,-1.66081870e+00,1.07725334e+00
,-1.16646320e-01,6.68500185e-01,-2.72882640e-01,9.60672259e-01
,-1.20111525e+00,-1.41703975e+00,1.66905117e+00,1.22745538e+00
,1.17302993e-02,-3.63293201e-01,1.39165437e+00,-1.43926769e-01
,1.12461281e+00,-1.08863664e+00,3.46102357e-01,7.51800388e-02
,-1.29072040e-01,2.65300179e+00,-8.04959476e-01,-6.86808407e-01
,-3.62184197e-01,6.47986382e-02,-5.59881330e-03,2.97925591e-01
,-1.98865339e-01,-7.13880181e-01,9.93024230e-01,-1.61429727e+00
,-1.64142704e+00,6.97395623e-01,-2.08026230e-01,1.87536025e+00
,-3.18071365e-01,-1.10548586e-01,-9.56378728e-02,5.97207904e-01
,-6.58443630e-01,1.51544690e+00,-1.02676654e+00,1.20206428e+00
,-2.00919211e-01,1.57578349e-01,1.00945845e-01,9.26668644e-01
,6.28653228e-01,-2.79278249e-01,6.77886248e-01,1.68880582e-01
,9.40699875e-01,-1.29843736e+00,-6.15832269e-01,3.45540702e-01
,-1.22514927e+00,3.03435296e-01,-2.46939510e-01,-1.24101043e+00
,-1.38681388e+00,-1.88349688e+00,9.39365029e-01,7.21985579e-01
,5.84275797e-02,3.96761298e-02,6.13313198e-01,-1.18720937e+00
,1.79351258e+00,7.85042703e-01,-1.14075267e+00,6.86704397e-01
,1.13140285e+00,2.02790403e+00,-4.20811400e-02,3.23806912e-01
,-1.87870753e+00,4.36505049e-01,2.55283475e+00,-1.62204289e+00
,-1.06860352e+00,5.49966872e-01,1.42782301e-01,-1.32069099e+00,   
]

request = match_service_pb2.MatchRequest()
request.deployed_index_id = DEPLOYED_INDEX_ID
for val in query:
    request.float_val.append(val)

response = stub.Match(request)

In [6]:
response

neighbor {
  id: "Amit Shah_92"
  distance: 44.124298095703125
}
neighbor {
  id: "Amit Shah_80"
  distance: 75.77590942382812
}
neighbor {
  id: "Amit Shah_19"
  distance: 80.10911560058594
}
neighbor {
  id: "Amit Shah_45"
  distance: 83.44609069824219
}
neighbor {
  id: "Amit Shah_12"
  distance: 83.97708892822266
}
neighbor {
  id: "Amit Shah_3"
  distance: 86.94171905517578
}
neighbor {
  id: "Amit Shah_37"
  distance: 87.36231994628906
}
neighbor {
  id: "Amit Shah_58"
  distance: 89.91946411132812
}
neighbor {
  id: "Amit Shah_67"
  distance: 91.34013366699219
}
neighbor {
  id: "Amit Shah_53"
  distance: 91.44570922851562
}

In [20]:
faces = []
for ne in response.neighbor:
    # print(ne)
    # print(ne.id, ne.distance)
    if ne.distance < 90.0:
        face_name_ = ne.id.split('_')[0]
        faces.append(face_name_)
# return the first entry
# print(faces)
face = [faces[0]]
print(face)

['Amit Shah']


### Batch Query

You can run multiple queries in a single RPC call using the BatchMatch API:

In [21]:

def get_request(embedding, deployed_index_id):
    request = match_service_pb2.MatchRequest(num_neighbors=1)
    request.deployed_index_id = deployed_index_id
    for val in embedding:
        request.float_val.append(val)
    return request

In [22]:
# Test query
# queries = [9816192,-0.7448069,-1.8027778,-1.7395629,0.16280416,1.2310203,-0.3618377,1.4659195,-0.6501849,1.0882517,0.10971829,-0.5627692,-0.15019046,-0.48551172,-0.41610557,0.5273846,-0.19850771,-0.3284484,0.4305416,1.0691893,-0.2723782,-0.09439357,0.16869284,-1.2967464,1.072695,-0.07779307,0.8316756,0.1254679,1.0575303,1.0772109,1.5256245,1.2180685,1.226398,-1.0671296,-0.86190045,1.1213456,-0.9838395,0.2509745,0.68849146,-1.9389365,0.38959187,-0.10003081,1.1011937,0.016147107,-1.7182889,0.3174295,-0.6336806,1.2639811,0.083934896,0.74825305,-0.77276206,1.618168,-0.7307955,0.3288172,-0.80723184,-0.24376033,-2.3887613,-0.44180405,0.4671563,0.035028607,-1.2847717,-0.13177945,-0.109141394],[0.18413678,0.14467666,1.1700193,-1.7318141,-1.6199924,1.6044966,-1.1750349,0.49471578,-0.1909078,1.2311367,0.07380035,0.14829472,1.1527169,0.35876116,-0.6423862,-0.49840707,1.3095233,-2.007389,1.6367638,0.1726163,0.5062605,0.019260604,0.31504264,1.3287826,-1.1997359,-0.46761233,-0.9253086,-1.5636103,-0.41088992,-0.008263953,0.65959084,-1.5495433,-0.39056176,1.2856237,0.44624385,1.1084298,-0.1120715,-0.5008984,1.3865032,-0.25656816,-0.08751078,-0.09833423,0.65624833,1.0134637,-0.66727865,0.039285205,-0.1820786,0.83991635,-0.3733094,-0.2259379,0.81437176,0.60329145,-1.1501639,-0.9840886,-1.365501,-0.21425706,0.121758,-1.3356931,-0.9387212,-1.7900875,-1.1644254,-0.65272635,-0.036951914,-0.16353361,-0.12553385,-0.9816192,-0.7448069,-1.8027778,-1.7395629,0.16280416,1.2310203,-0.3618377,1.4659195,-0.6501849,1.0882517,0.10971829,-0.5627692,-0.15019046,-0.48551172,-0.41610557,0.5273846,-0.19850771,-0.3284484,0.4305416,1.0691893,-0.2723782,-0.09439357,0.16869284,-1.2967464,1.072695,-0.07779307,0.8316756,0.1254679,1.0575303,1.0772109,1.5256245,1.2180685,1.226398,-1.0671296,-0.86190045,1.1213456,-0.9838395,0.2509745,0.68849146,-1.9389365,0.38959187,-0.10003081,1.1011937,0.016147107,-1.7182889,0.3174295,-0.6336806,1.2639811,0.083934896,0.74825305,-0.77276206,1.618168,-0.7307955,0.3288172,-0.80723184,-0.24376033,-2.3887613,-0.44180405,0.4671563,0.035028607,-1.2847717,-0.13177945,-0.109141394],[-0.6527325,-0.46002162,-0.17663276,-0.81140214,-1.032406,0.54161894,-0.4328485,0.36944166,0.6905523,-0.20041354,0.6864796,0.681825,0.9503024,1.0567664,0.11691467,-0.64779764,-0.9121188,-0.6323731,0.20667775,1.2265365,0.09392789,-0.530397,0.21178347,0.69412977,2.0142603,1.5104163,0.28759924,0.44440737,0.11779354,-1.0549424,0.0741029,0.30513865,-0.17530659,0.64300656,-0.7095175,0.5820626,-0.44098482,-0.44718242,0.29324716,-0.053111784,-0.010820376,0.6508963,-0.46727577,-1.4214003,-0.5304358,-0.39478037,-0.21847266,-0.64085114,-1.1323403,-0.2172702,-0.31059074,0.70603216,-0.32577443,-0.8887467,-0.60617787,0.473167,1.2807322,0.32923022,0.78128844,0.18720192,1.943259,-0.072848186,0.98476636,-0.70947444,-0.39064202,-1.6843693,0.054231454,-0.91281646,0.23715061,-1.3312396,0.20295969,-0.13792513,-0.38381797,0.92857844,0.4874856,-0.7459441,0.75593007,-0.55905926,0.399276,-0.057654556,0.36737126,-0.5129546,-0.6241364,0.17834072,-0.15344253,0.506616,1.0232884,0.5740213,0.6606432,0.48459333,-1.4331911,-1.6764264,0.4804465,0.772825,0.58467114,1.0799781,-0.6199612,0.43116918,0.3785324,-0.11021599,0.74899054,-0.47892073,1.2604288,-0.5757888,0.36970234,-0.148336,-0.57367164,0.318303,0.090885356,-0.884959,1.7383541,-0.26257086,0.56107926,-0.030965535,1.626397,0.6418035,-0.03811022,0.53946364,0.68184286,1.186307,-0.13145073,0.6319256,-1.2999176,0.53756,-0.27358598,1.1925207,0.13187881,-0.035568252],[0.21685691,-0.74236,0.5813849,0.31675515,-1.6762481,2.5330088,-1.0266,0.40983456,0.47244725,0.413979,-0.5419996,0.75381154,1.9768379,0.10541016,-0.37758306,-0.6771785,1.5422283,-0.614758,1.6940457,0.9952977,-0.18833739,0.24632266,0.8098842,0.9005453,-0.47566104,-0.0015658513,-0.2949663,-0.14152941,-0.54262865,-0.6222423,0.18675938,-0.6789349,0.01172789,0.39422706,-0.23747936,-0.13254286,-1.3294531,-0.2114808,0.05801169,-0.065232776,-0.31281516,-0.18682103,0.38905686,0.002024293,-0.027333027,0.16223763,0.494871,-0.43051964,-0.15713158,0.789883,0.30112478,-0.8198618,-0.7788522,-1.3538176,0.45150644,0.22904247,0.28686756,-1.1262417,-0.38479263,-0.94411075,-0.108317934,-0.5213486,1.5195915,-0.33720872,-0.72543025,-1.4418588,-0.7354839,-1.1261866,-1.3995473,-0.447938,0.17401975,-0.4766734,1.4919164,0.14210448,1.6950531,0.18311928,-0.93880665,-0.15865886,0.191989,0.664943,0.5422651,-0.6143166,-0.93413764,0.058260456,1.3316399,-0.058983356,0.6745224,0.25968295,-0.8560741,0.35622123,0.17840774,0.17545128,-0.3987922,0.3876696,1.0912793,0.36760116,0.08524214,1.8528432,-1.3554859,-1.3086898,1.344731,-0.64518857,0.942451,0.53502345,-0.4113335,-0.81055653,-0.28722066,0.45935863,-0.25571945,-2.1783602,0.9379564,0.15091206,-0.2250387,0.88230395,-0.33675513,-1.0246297,0.49273238,-0.08585124,0.78423107,0.13077068,-0.246471,-1.3224143,-0.5338822,0.8675535,-0.027134344,-0.532081,-0.51900625,0.6919282],[-0.09470743,-0.9957243,-0.010976996,0.0803454,0.335115,0.7842994,-0.12350471,1.0259149,1.2593132,0.24778858,0.55998015,0.018294046,1.2738311,0.56743026,-1.0193995,-0.87597585,-0.5192492,-0.37678283,0.6551954,0.7200092,-0.3619045,0.05566327,-0.74476004,1.518461,1.1577336,0.59153545,0.36944008,0.047567844,-0.18735571,-1.3357652,0.29635555,-0.043123763,-0.5242069,0.048581295,-0.91532797,0.5513883,-0.9868867,-0.09840646,-0.0066524595,0.52178425,0.25186944,0.36972278,0.8738141,0.07670653,-0.36123303,-0.15546341,-0.63910425,0.1574087,-0.26002997,-0.7703928,-0.6891219,1.3843614,-1.0664507,-0.5086682,-0.8237348,1.0254394,1.0091256,-1.2604334,-0.39869818,0.42056027,0.90474033,1.213972,0.3922502,0.13676052,-0.044360906,0.13491757,-0.2519035,-0.15900078,-0.49683252,-0.5554099,0.3285706,0.016492829,0.3485614,0.53839874,1.0599217,-0.6861948,-0.115299664,-0.13781114,-0.73526734,1.1489714,0.5011151,-0.56288743,-0.33891213,0.030729627,0.54306877,0.058530353,-0.01975809,0.35059577,0.6704149,0.3758355,-0.08324405,-0.33034164,0.23877743,0.6330583,0.7342317,1.0174036,-0.35716024,0.3647527,-0.42698702,-0.319528,0.76883364,-0.627468,0.72778904,-1.0153054,0.15626603,-0.23017238,-0.0034312205,-0.37003124,0.2094546,-1.4682338,1.677014,-0.36502475,0.38025546,0.04337758,0.7213974,0.52587414,-0.23969233,1.2773775,0.77503777,0.7514368,-0.10919753,0.28309998,-0.63711935,0.5232253,0.21996261,0.00026608258,0.23745285,0.00061229244],[0.3515678,-1.634084,-0.15391451,-0.66750586,-2.040142,1.5595897,-2.1159961,0.32631233,0.60384107,1.0031524,-1.0047858,0.20731995,0.45701063,0.97612274,-0.7701458,-0.15855943,1.8689562,-0.53064585,1.2202824,-0.15252145,0.15184662,-0.3736378,1.5102271,1.7953545,-0.20195398,-0.39792484,-0.9771092,-1.3974319,1.3956796,0.43259653,1.247536,-1.4074383,-0.39765146,1.1410106,0.1711696,0.35628036,-1.8344634,1.0232233,0.2524213,0.042367067,-0.5928182,-0.44811085,1.6109681,0.5388657,0.99062717,-0.7053302,1.2997863,0.13902575,-0.99954116,0.44389668,-0.19396515,0.43770856,-1.5195844,-1.1511279,-1.1326722,0.92540425,0.72627175,-0.5062168,-0.32070747,-0.23028067,-0.3274488,-0.61668164,0.65832114,-0.07312229,-0.35960928,-0.71956366,-0.3567589,-0.06299948,-2.1067898,-1.3199012,0.21221896,-0.4569715,0.9166688,-0.50993437,2.1080365,-0.28577587,-1.1557136,-1.7777491,-1.4655622,-1.7181861,0.44727966,0.012890211,-0.020539634,0.58306766,0.34716433,-0.40816557,-0.42915985,0.31261933,-0.80986464,-0.14053644,-0.30899698,-0.40045553,0.52384543,0.24726394,0.8177648,1.1738976,-0.770011,1.6275287,-0.6742334,-0.63537216,1.110465,-0.5080552,0.9504686,0.30950323,-0.8092762,-0.69433594,-0.87024385,1.5524361,0.10499555,-2.6537018,0.83975315,0.57519656,0.65740883,0.09023847,0.12553553,-2.0168543,1.6492207,-0.0008898154,-0.5718152,-0.5388767,1.0601186,-1.1574098,-0.016015552,0.20193635,-0.08900456,-1.059496,-0.8500817,-0.5618607],[0.24820755,-0.13060969,0.21212643,-0.14288382,-0.2884269,2.333124,-0.6668295,0.18279135,0.059188843,0.35928446,-0.72351176,0.22057416,1.5879958,0.2770859,-0.00081653893,-0.9332528,0.50624967,0.1536665,1.1448402,1.7614877,0.24595991,1.0020515,-0.46187165,0.74953425,0.7263755,0.47215962,-0.68761814,0.5042121,-0.34581268,-0.7705524,0.7807412,-0.41430333,-0.39252847,0.2083332,-0.94408655,-0.3971553,-0.5584936,-0.40949345,-0.47901678,-0.5518322,-0.0680359,-0.3174044,0.7138871,-0.19822884,-0.33166778,-0.49479228,-0.21129082,-0.4700257,0.020489722,-0.60382223,0.17194471,-0.31990975,-0.7897453,-1.1185282,-0.55046546,0.12838268,0.42841578,-1.8105831,0.020382755,-0.601147,0.455275,0.8269739,1.2450651,0.6148923,-1.1102371,-1.2292726,0.06545773,-1.4876488,-0.21257508,-0.1805548,0.63876814,-0.6416253,1.1085165,0.31301367,1.1446965,0.057441488,0.0182534,-0.41614828,0.37311
queries = [[2.118546962738037, -0.013021700084209442, -0.3258954584598541, -0.5392261147499084, -2.2430038452148438, 1.4208695888519287, 0.11820642650127411, 0.6939780712127686, -0.0448530875146389, 0.5809880495071411, -0.3057672083377838, -0.2048531174659729, -0.02443012222647667, -0.30874377489089966, -0.6117008924484253, 0.23289434611797333, 1.1948736906051636, 0.38890576362609863, 0.9329477548599243, -0.5442702770233154, 0.08512429893016815, -1.4713857173919678, 1.2732497453689575, 0.30391740798950195, 0.5498077869415283, -1.402136206626892, -0.9531724452972412, -0.9746171236038208, -0.5040358901023865, 0.4149115979671478, -0.06733724474906921, -2.4312832355499268, 0.6912112236022949, 0.5042687654495239, 0.48167726397514343, -1.4445549249649048, -0.35301682353019714, 1.9175089597702026, 0.34120893478393555, 0.9329705238342285, -1.791138768196106, 0.05788864940404892, 0.39945462346076965, -1.0653802156448364, 1.1175309419631958, -0.08843955397605896, -0.551444411277771, 2.8107261657714844, 1.60438072681427, -1.0804386138916016, 0.2638697922229767, -1.1440562009811401, 0.15228256583213806, -0.6644552946090698, 0.6492002606391907, 0.6822055578231812, 0.012730310671031475, 0.12283693253993988, 1.9982033967971802, 0.02643435448408127, -0.3810011148452759, 1.9324805736541748, -0.01996026560664177, 0.6866852641105652, 0.2508283853530884, 0.5875520706176758, -1.801210880279541, 0.2674277424812317, -0.6264219880104065, 0.09610942006111145, -0.021816473454236984, -0.6362850666046143, 0.36334314942359924, -1.277477741241455, 0.8642569780349731, 0.012324061244726181, -0.5763729810714722, 1.006130576133728, 0.012655064463615417, -0.7897902131080627, -0.46759817004203796, -1.5538944005966187, -1.590641736984253, -1.2782098054885864, 2.081791877746582, -0.4090043902397156, -0.6438973546028137, -0.871679961681366, -1.047347068786621, 0.09694898128509521, 1.4852601289749146, 1.4078847169876099, 0.6480395197868347, 0.3184085190296173, -0.6514943242073059, 0.5482276082038879, 0.7690707445144653, 0.1989520788192749, -1.2540867328643799, 0.6331718564033508, 0.6476159691810608, -0.22038942575454712, 2.2349696159362793, 0.8375366926193237, -1.1868784427642822, 0.5579397678375244, -0.5884110331535339, -0.21139860153198242, -1.9572135210037231, -0.08576033264398575, 0.8261260390281677, -0.12310189008712769, -2.1986277103424072, -0.5441850423812866, -1.4979476928710938, 0.002759777009487152, -0.32307133078575134, 0.8101828694343567, -0.07837999612092972, 0.26630929112434387, 0.67682945728302, -0.3607063591480255, -0.6590176820755005, -0.170370951294899, -0.8792850375175476, -0.9447299838066101, -0.719833254814148, -1.4162365198135376], [-0.198530912399292, -1.4971776008605957, 1.5973610877990723, -0.2951831519603729, -0.9029490351676941, -0.06686321645975113, 0.3552708029747009, -0.11342212557792664, 1.7126306295394897, -0.09207148849964142, -0.07844021916389465, -1.2351651191711426, 1.3947306871414185, -1.9453275203704834, 1.1068106889724731, 1.0075297355651855, 0.5453153848648071, 0.9206985831260681, 0.8569179177284241, -0.7873399257659912, 0.4587508738040924, -1.3132740259170532, 0.14322099089622498, 0.5921074748039246, -2.0670690536499023, -1.0416853427886963, 1.3448716402053833, -0.16108520328998566, -0.8483697175979614, 0.432079941034317, -2.932401657104492, 2.459688663482666, -0.8258686661720276, 0.8686158657073975, 0.5511044859886169, -1.783318281173706, 0.07396721839904785, 0.8735816478729248, 0.44333577156066895, 0.3530426323413849, 1.6633132696151733, -0.7063894867897034, 1.7964600324630737, -0.603463351726532, -0.20789773762226105, -0.0013007745146751404, 0.20619869232177734, -0.7750699520111084, 0.5883492827415466, 0.15019279718399048, 0.9311829209327698, -1.5431551933288574, -0.9287973642349243, 1.718420147895813, 0.6573407053947449, -2.3996243476867676, -1.0414104461669922, -0.46951478719711304, -1.6005750894546509, -0.43704429268836975, -0.22813653945922852, -0.28153133392333984, -0.6000317931175232, 0.6324875950813293, -0.04009172320365906, 1.0484422445297241, 1.063758373260498, -0.6588342785835266, -0.5536823272705078, -0.3488430678844452, -0.8003819584846497, -1.047927975654602, 2.413529872894287, 0.12537658214569092, 0.4963311553001404, 0.19660258293151855, -1.6017919778823853, 0.26028430461883545, -0.5735283493995667, -0.8553931713104248, 0.13695591688156128, 0.38814854621887207, -0.9865179657936096, -1.434780240058899, -1.2059746980667114, -0.8359695076942444, 1.4138166904449463, -1.3528293371200562, -2.0466487407684326, 0.5743588805198669, 0.5912349820137024, 1.1902648210525513, -0.17383931577205658, -0.6520994901657104, 0.3107731342315674, 0.5590965747833252, -0.32951074838638306, 1.3114382028579712, -2.4081573486328125, -0.3655891418457031, -0.21614350378513336, -1.2750006914138794, -0.7606200575828552, 0.7481397986412048, 0.3057936131954193, 0.7495997548103333, -1.2548664808273315, -0.48496660590171814, -0.785793662071228, 1.1268714666366577, 0.6837136149406433, -1.5399882793426514, -0.5507254600524902, -0.7333093285560608, -3.049243927001953, -1.021457314491272, -1.2845735549926758, -1.9448356628417969, -1.2638278007507324, -0.8127043843269348, -2.459954261779785, -1.261998176574707, 0.9758577346801758, 0.39707592129707336, -0.3943692445755005, 1.6295435428619385, 0.0942917913198471, 0.2299695461988449], [0.5144336223602295, -0.59465491771698, 0.47790029644966125, -1.124071478843689, 0.35987237095832825, 0.003985036164522171, 1.7364193201065063, -1.0399117469787598, 0.5072877407073975, -0.12367112934589386, 0.019508369266986847, 0.38440099358558655, -0.08985165506601334, 0.558857798576355, -1.9376187324523926, -0.23774568736553192, 0.014065008610486984, -0.3597065806388855, 1.2063082456588745, -0.8582608103752136, 0.7260318994522095, -0.17837077379226685, -2.3147943019866943, -0.22845405340194702, 0.2804839015007019, 0.13565686345100403, 0.9515142440795898, -1.3903874158859253, -0.8552996516227722, 0.4552302062511444, 0.8516110181808472, 1.1382625102996826, 0.19221821427345276, -1.3605124950408936, -0.946489155292511, 0.533363938331604, -0.4780927002429962, -0.14964741468429565, 1.0463478565216064, 0.8830106258392334, 0.8109706044197083, 0.4627421200275421, 0.7363053560256958, 0.31305429339408875, 1.2644541263580322, 0.7847429513931274, -2.9059813022613525, 1.7139768600463867, 0.40776515007019043, 1.0125055313110352, -1.0610730648040771, 1.0477163791656494, -0.9247443675994873, -0.8405344486236572, 0.5040292143821716, 0.566049337387085, -0.0708429366350174, -0.9262195825576782, 0.20068879425525665, -0.3464980721473694, -0.5217214822769165, 0.49184268712997437, -0.6021115779876709, 0.40098708868026733, -0.03209974244236946, 1.8631941080093384, -0.6199786067008972, -0.1612669676542282, -1.1451913118362427, 0.6311489343643188, 0.47913941740989685, 0.5891664028167725, 1.0419557094573975, -1.0672739744186401, 0.7586084604263306, -2.2983269691467285, -0.6192686557769775, -0.20444129407405853, -1.4866111278533936, 1.6216304302215576, -0.9838666319847107, 1.008747935295105, 0.36034002900123596, -1.2295893430709839, 0.7136270403862, 1.254457712173462, -1.8552011251449585, 0.6522489786148071, 0.13948191702365875, 0.442878395318985, 0.7477810382843018, 1.6907156705856323, 1.059187889099121, 0.5934590697288513, 1.269315242767334, 0.25490739941596985, 0.7093484401702881, 0.860188901424408, -1.3150415420532227, 0.07242853939533234, 0.8096097111701965, -0.8270384073257446, 1.3206416368484497, -1.5316851139068604, -1.8607800006866455, -1.1147254705429077, 1.5314866304397583, 1.696233868598938, -0.06571582704782486, -0.6149551272392273, 0.3895835280418396, -1.2106913328170776, 1.2171562910079956, 0.34067144989967346, -0.03507301211357117, 0.5035238265991211, 2.2273781299591064, 2.600999355316162, 0.0824340432882309, -0.1798509806394577, 0.7013037204742432, 1.0687730312347412, 2.223860263824463, -1.9923282861709595, -0.6272052526473999, -0.5817416906356812, -0.8585206866264343, 0.16398993134498596], [-0.5305699110031128, -0.33019667863845825, 0.2535208463668823, -0.6413379311561584, -1.3746345043182373, 1.1445281505584717, -0.30136534571647644, 0.5398967266082764, 0.850663959980011, -0.6012423634529114, -0.23572462797164917, 0.6171404123306274, 1.8305258750915527, 0.19519701600074768, 0.10607700794935226, -0.6423410177230835, 0.6516464948654175, -1.931764841079712, 0.7364111542701721, 0.3943420946598053, -0.7979141473770142, -0.04878056049346924, -0.2171218991279602, 1.6514456272125244, 0.5521128177642822, 1.1352639198303223, 1.5091689825057983, 0.38719630241394043, -0.5386963486671448, -0.9906826615333557, -0.1617065817117691, -0.2070281058549881, 0.1418989598751068, 0.7416083216667175, -0.21564678847789764, 0.35868388414382935, -0.6620075106620789, -0.5156772136688232, 0.34613722562789917, 0.09778879582881927, -0.4159148633480072, 0.6268783807754517, -0.39277318120002747, -0.4102693498134613, -0.5634706020355225, -0.00540882907807827, 0.3322504758834839, -0.4194912910461426, -0.3778737783432007, 0.37787529826164246, -0.31309938430786133, 0.8011848330497742, 0.4268491566181183, -1.1408991813659668, 0.27344703674316406, 0.7439393997192383, 0.37839242815971375, -0.941891610622406, 0.1270265430212021, -0.41752979159355164, 0.36881300806999207, 0.6547180414199829, 0.3418942391872406, 0.44005951285362244, 0.00015724264085292816, -0.22047929465770721, -0.5934036374092102, -0.04990116134285927, -0.7568756937980652, 0.08951980620622635, 0.7472455501556396, -0.12477520108222961, 1.2424392700195312, -0.6979236602783203, 0.36503785848617554, 0.2569558918476105, -0.5571841597557068, -0.2717623710632324, 0.2073078453540802, -0.34325188398361206, -0.1524493247270584, -0.33216115832328796, -0.6235193610191345, 0.5896240472793579, 1.991883635520935, -0.2515711486339569, 1.2989733219146729, -0.053672611713409424, -0.38623565435409546, 0.2321939766407013, -0.11673697084188461, -0.463491827249527, -0.1817193180322647, -0.0601755753159523, 1.0534652471542358, 1.5707383155822754, -0.07083591818809509, 1.386231541633606, -1.0352232456207275, -0.38862869143486023, 1.7287745475769043, 0.09151238948106766, 0.9762588739395142, 0.0351107120513916, -0.6392652988433838, 0.4573873281478882, -0.8318305611610413, 0.4427644610404968, -0.11726007610559464, -1.6157464981079102, 0.761758029460907, -0.31704235076904297, 0.4964231252670288, 0.342804878950119, 0.049234721809625626, 0.7975654602050781, -0.7893702983856201, 0.5739270448684692, 1.4190927743911743, 0.25733527541160583, 0.9395102262496948, -0.7338955402374268, -0.37872421741485596, 0.15401235222816467, 0.9718120694160461, -0.8590283989906311, 1.26738703250885, 0.19586573541164398], [-0.3542868494987488, -0.6068665981292725, 0.008800394833087921, -0.32833331823349, -0.539149820804596, 1.7358015775680542, -0.27205127477645874, -0.04250272735953331, -0.1616007387638092, 0.012407023459672928, -0.4978909492492676, 0.5853409171104431, 1.39695143699646, 0.3450048565864563, 0.12396567314863205, -0.25606560707092285, 0.586153507232666, -0.7410386800765991, 0.9868607521057129, 0.7649319767951965, -0.0323554128408432, 0.07042951136827469, 0.058266349136829376, 0.8268976807594299, 0.32074859738349915, 0.3506060540676117, -0.14635908603668213, -0.008771494030952454, 0.2259146124124527, -0.5829467177391052, 0.7998309135437012, -0.05787966027855873, 0.3712269365787506, 0.5361840724945068, -0.5139668583869934, -0.5763315558433533, -0.6417669653892517, -0.28915584087371826, -0.7026141881942749, -0.017468877136707306, 0.18783405423164368, 0.4957095980644226, 0.5357110500335693, 0.44285356998443604, -0.21409639716148376, -0.5053372383117676, 0.34077388048171997, -0.25472766160964966, 0.43350207805633545, 0.03843454644083977, -0.2654663920402527, -0.15434511005878448, -0.5162573456764221, -1.1263906955718994, 0.09327857196331024, -0.18407461047172546, 0.2847427427768707, -0.255760133266449, -0.2948741912841797, -0.7827463746070862, 0.10024946182966232, 0.20795206725597382, 0.3832985758781433, 0.37954869866371155, -1.3477140665054321, -0.09210792183876038, -0.0275685116648674, -0.15524695813655853, 0.30998602509498596, -0.29387593269348145, 0.39953622221946716, -0.6679903268814087, 0.6855114102363586, -0.06282452493906021, 0.5871788263320923, 0.14554539322853088, -0.10746675729751587, -0.24218963086605072, -0.011273197829723358, 0.5885227918624878, -0.18134242296218872, -0.07179653644561768, -0.7507853507995605, 0.17742787301540375, 0.8295885324478149, 0.6008156538009644, 0.7434350848197937, 0.031133726239204407, -0.40947431325912476, 0.5786612629890442, 0.21496708691120148, -0.5518674254417419, -0.06961777806282043, 0.31615570187568665, 1.1596096754074097, 0.6475501656532288, -0.08930820226669312, 0.895336925983429, -1.0077121257781982, -0.3652937412261963, 0.6398051977157593, -0.38308829069137573, 0.4865638315677643, 0.42202359437942505, -0.5320329070091248, -0.6071283221244812, -0.24434319138526917, -0.06738580763339996, -0.3581032454967499, -1.0059804916381836, -0.19969835877418518, 0.3600369691848755, 0.18672901391983032, 0.44282346963882446, -0.296598345041275, -0.32643556594848633, -0.168923020362854, -0.38945090770721436, 0.39194077253341675, -0.2324301153421402, 0.4209361672401428, -0.07807242125272751, -0.1651330292224884, 0.4090646803379059, 0.6133414506912231, -0.9030742049217224, 0.40842685103416443, 0.7680414915084839], [-0.754199743270874, -0.04091925173997879, -0.21093088388442993, -0.5510718822479248, -1.2415542602539062, 0.10782011598348618, 0.12124928832054138, -0.027931205928325653, -0.10091311484575272, -0.6432794332504272, -0.6369632482528687, 0.6623658537864685, 1.098237156867981, 0.10688206553459167, -0.050871238112449646, -0.297288715839386, 0.6090244054794312, -1.1782479286193848, 0.8098786473274231, -0.17987772822380066, -1.0396358966827393, 0.01692577451467514, -0.39364463090896606, 0.40717729926109314, 0.0559186153113842, 0.9156891703605652, 1.5321341753005981, 0.2168649286031723, -1.1006020307540894, -1.199242115020752, -0.9414522647857666, -0.015730958431959152, 0.004679580219089985, 0.6157509684562683, 0.5472960472106934, 0.3147388696670532, 0.4034566581249237, -0.8117131590843201, 0.47808200120925903, -0.19596412777900696, -0.8393664956092834, 1.0190106630325317, -0.6614464521408081, 0.6219386458396912, -0.20146895945072174, -0.5177900195121765, 0.07930907607078552, -0.10698007047176361, 0.10967366397380829, 0.2510651648044586, -0.7795004844665527, 0.12243099510669708, 0.0915527492761612, -0.18186384439468384, 0.22939294576644897, 0.6498721837997437, 0.7887111306190491, -0.04820959270000458, -0.6435135006904602, -0.7748964428901672, -0.4751584529876709, 0.8482090830802917, -0.23237672448158264, 0.3751520812511444, -0.11967606097459793, 1.1533833742141724, -0.23248150944709778, 0.08441942185163498, -0.03260229155421257, 0.3633415699005127, 0.743831992149353, 0.515546441078186, 0.3171965181827545, -0.7995901107788086, 0.16436392068862915, 0.047069355845451355, -0.905243992805481, -0.4311322271823883, 0.5822123885154724, -0.6567810773849487, -0.09975215792655945, -0.38171786069869995, -0.4416850805282593, 0.6209403276443481, 1.7051647901535034, -0.11688282340765, 0.8827836513519287, 0.1059018075466156, -0.38145357370376587, 0.6326279640197754, 0.05912336707115173, -0.5596586465835571, -0.39338651299476624, -0.41316327452659607, 0.009378280490636826, 1.679794192314148, 0.17646034061908722, 0.3797413408756256, -0.4720768630504608, 0.11218937486410141, 0.9777904152870178, 0.3695025146007538, 0.5685186386108398, -0.23449482023715973, 0.10622503608465195, 0.09399250894784927, -0.7607733011245728, 0.07221835851669312, -0.20425574481487274, -1.1877741813659668, 0.5740727782249451, -0.45196110010147095, 0.41525065898895264, 0.9041554927825928, -0.5133509635925293, 0.9637027978897095, -0.8703827857971191, 0.8074366450309753, 1.521902322769165, 0.2772832214832306, 0.6567826271057129, -0.6189382076263428, -0.23602907359600067, 0.054919078946113586, 1.0521354675292969, -0.8921195268630981, 1.4162052869796753, -0.36796849966049194]]

batch_request = match_service_pb2.BatchMatchRequest()
batch_request_ann = match_service_pb2.BatchMatchRequest.BatchMatchRequestPerIndex()
batch_request_brute_force = (
    match_service_pb2.BatchMatchRequest.BatchMatchRequestPerIndex()
)
batch_request_ann.deployed_index_id = DEPLOYED_INDEX_ID
#batch_request_brute_force.deployed_index_id = DEPLOYED_BRUTE_FORCE_INDEX_ID
for query in queries:
    batch_request_ann.requests.append(get_request(query, DEPLOYED_INDEX_ID)
    #batch_request_brute_force.requests.append(
        #get_request(query, DEPLOYED_BRUTE_FORCE_INDEX_ID)
    )
batch_request.requests.append(batch_request_ann)
#batch_request.requests.append(batch_request_brute_force)

response = stub.BatchMatch(batch_request)
response

responses {
  deployed_index_id: "ann_living_media_deployed"
  responses {
    neighbor {
      id: "Mithali Raj_687"
      distance: 12.278135299682617
    }
  }
  responses {
    neighbor {
      id: "Sidharth Malhotra_623"
      distance: 66.82612609863281
    }
  }
  responses {
    neighbor {
      id: "Amit Shah_53"
      distance: 17.26312255859375
    }
  }
  responses {
    neighbor {
      id: "Mithali Raj_747"
      distance: 44.14092254638672
    }
  }
  responses {
    neighbor {
      id: "Mahendra Singh Dhoni_428"
      distance: 23.116268157958984
    }
  }
  responses {
    neighbor {
      id: "Mahendra Singh Dhoni_428"
      distance: 49.259498596191406
    }
  }
  status {
  }
}

In [26]:
faces = []
# dir(response)
# for res in response.responses:
# for neig in response.responses[0].responses:

for neig in response.responses[0].responses:
    # print(neig.neighbor[0].id, neig.neighbor[0].distance)
    face_id = neig.neighbor[0].id
    dist = neig.neighbor[0].distance
    if dist < 30.0:
        face_name_ = face_id.split('_')[0]
        faces.append(face_name_)

        
print(faces)

['Mithali Raj', 'Amit Shah', 'Mahendra Singh Dhoni']


In [69]:
query=[[1.1782426,1.0406778,-0.44876,-0.7633594,-1.849223,2.2980382
,0.43171912,-0.5249385,1.6232405,-0.5861312,-0.71334064,0.6076759
,0.14184979,1.3373281,-0.49538466,-0.5900156,-0.9048157,-0.8203462
,1.0834305,0.97602504,0.81175625,-1.5748513,0.38024062,-0.78434944
,0.07006576,-0.9121722,-0.19874564,-1.4220611,-1.2230803,-0.9197947
,-0.7203435,-0.9375914,-0.09938783,0.8080363,-0.754516,-0.17224674
,0.53428876,-1.3720698,1.6864536,-0.12772012,1.6394142,1.311719
,1.2664603,-0.6348907,-1.5060147,-0.24698094,-1.0028895,-1.5993083
,-0.21794772,-0.70590544,-0.99432963,-0.02272956,0.03609939,-1.8941488
,0.6971372,-0.33864325,2.17427,1.2251923,1.2888476,-0.14010504
,1.5132291,0.95102006,0.5111402,1.2493731,0.6373472,0.993629
,1.2477914,-0.46476325,-0.47406006,0.39714545,0.92181164,0.98808897
,0.35596132,-0.4125048,1.276678,1.0622914,-0.14278081,0.26561818
,0.49647754,-0.4398989,0.6308249,1.5569562,1.766642,-1.63171
,-0.17620248,-1.051231,-1.3411899,0.6125753,-1.4611464,0.48370305
,-0.15419513,-1.6263596,-0.97733307,1.1078069,-1.0737462,0.16510986
,-2.1010196,0.7841323,-0.9470381,1.0418843,-0.41200632,-1.708659
,-0.8174982,1.3656204,-0.4826603,-0.89417154,0.28905457,1.0995983
,-1.6365868,0.32411125,1.2217121,0.71487683,0.9310758,0.6489747
,-1.2615271,-0.2319146,0.4160086,0.17292742,1.4088401,0.7208057
,1.6679052,-0.67300296,0.7815629,-0.915559,-0.23436396,-1.5972084
,1.0581964,-0.7234948,],

 ]


batch_request = match_service_pb2.BatchMatchRequest()
batch_request_ann = match_service_pb2.BatchMatchRequest.BatchMatchRequestPerIndex()
batch_request_brute_force = (
    match_service_pb2.BatchMatchRequest.BatchMatchRequestPerIndex()
)
batch_request_ann.deployed_index_id = DEPLOYED_INDEX_ID
#batch_request_brute_force.deployed_index_id = DEPLOYED_BRUTE_FORCE_INDEX_ID
for query in queries:
    batch_request_ann.requests.append(get_request(query, DEPLOYED_INDEX_ID)
    #batch_request_brute_force.requests.append(
        #get_request(query, DEPLOYED_BRUTE_FORCE_INDEX_ID)
    )
batch_request.requests.append(batch_request_ann)
#batch_request.requests.append(batch_request_brute_force)

response = stub.BatchMatch(batch_request)
response

responses {
  deployed_index_id: "ann_living_media_deployed"
  responses {
    neighbor {
      id: "Mithali Raj_687"
      distance: 12.278135299682617
    }
  }
  responses {
    neighbor {
      id: "Sidharth Malhotra_623"
      distance: 66.82612609863281
    }
  }
  responses {
    neighbor {
      id: "Amit Shah_53"
      distance: 17.26312255859375
    }
  }
  responses {
    neighbor {
      id: "Mithali Raj_747"
      distance: 44.14092254638672
    }
  }
  responses {
    neighbor {
      id: "Mahendra Singh Dhoni_428"
      distance: 23.116268157958984
    }
  }
  responses {
    neighbor {
      id: "Mahendra Singh Dhoni_428"
      distance: 49.259498596191406
    }
  }
  status {
  }
}

In [40]:
print(response)

neighbor {
  id: "Amit Shah_92"
  distance: 44.124298095703125
}
neighbor {
  id: "Amit Shah_80"
  distance: 75.77590942382812
}
neighbor {
  id: "Amit Shah_19"
  distance: 80.10911560058594
}
neighbor {
  id: "Amit Shah_45"
  distance: 83.44609069824219
}
neighbor {
  id: "Amit Shah_12"
  distance: 83.97708892822266
}
neighbor {
  id: "Amit Shah_3"
  distance: 86.94171905517578
}
neighbor {
  id: "Amit Shah_37"
  distance: 87.36231994628906
}
neighbor {
  id: "Amit Shah_58"
  distance: 89.91946411132812
}
neighbor {
  id: "Amit Shah_67"
  distance: 91.34013366699219
}
neighbor {
  id: "Amit Shah_53"
  distance: 91.44570922851562
}



In [15]:
print(type(response))

<class 'match_service_pb2.BatchMatchResponse'>


### Compute Recall

Use deployed brute force Index as the ground truth to calculate the recall of ANN Index:

In [48]:
def get_neighbors(embedding, deployed_index_id):
    request = match_service_pb2.MatchRequest(num_neighbors=k)
    request.deployed_index_id = deployed_index_id
    for val in embedding:
        request.float_val.append(val)
    response = stub.Match(request)
    return [int(n.id) for n in response.neighbor]

In [49]:
# This will take 5-10 min

recall = sum(
    [
        len(
            set(get_neighbors(test[i], DEPLOYED_BRUTE_FORCE_INDEX_ID)).intersection(
                set(get_neighbors(test[i], DEPLOYED_INDEX_ID))
            )
        )
        for i in range(len(test))
    ]
) / (1.0 * len(test) * k)

print("Recall: {}".format(recall))

NameError: name 'test' is not defined

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [ ]:
index_client.delete_index(name=INDEX_RESOURCE_NAME)
index_client.delete_index(name=INDEX_BRUTE_FORCE_RESOURCE_NAME)

In [ ]:
index_endpoint_client.delete_index_endpoint(name=INDEX_ENDPOINT_NAME)